In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Linear regression with sklearn API

The objective of this notebook is to demonstrate how to build a linear regression model with sklearn. We will be using the following set up:
1. Dataset: California housing
2. Linear regression API: LinearRegression
3. Training: `fit` (normal equation) and `cross_validate`(normal equation with cross validation).
4. Evaluation: `score`(r2 score) and `cross_val_score` with different scoring parameters.

We will study the model dianosis with `LearningCurve`  and learn how to examine the learned model or weight vector.


In [2]:
# Importing packeges for manipulation and visualization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# importing dataset
from sklearn.datasets import fetch_california_housing
from sklearn.linear_model import LinearRegression# linear regression api

# importing models
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit#split in cross_validation

# importing metrics
from sklearn.metrics import mean_squared_error

# importing pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [3]:
np.random.seed(306)# setting seed to get repeatability across diff. runs
plt.style.use('seaborn')

We will use ShuffleSplit cross validation with:
* 10 folds(n_splits) and
* set aside 20% examples as test examples(test_size)

In [4]:
shuffle_split_cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=0)
# create 10 folds through shuffle split by keeping aside 20% examples as  test in each fold.

# STEP #1: Load the dataset

In [5]:
features, labels = fetch_california_housing(as_frame=True, return_X_y=True)

In [6]:
print("Shape of feature matrix:", features.shape)
print("Shape of label vector:", labels.shape)

Shape of feature matrix: (20640, 8)
Shape of label vector: (20640,)


As a sanity check, make sure that the number of rows in feature matrix and labels match.

In [7]:
assert (features.shape[0] == labels.shape[0])
print(features.shape[0])
print(labels.shape[0])

20640
20640


# STEP #2: Data exploration

# STEP #3: Preprocessing and model building

## 3.1 Train and test split

The first step is to split the training data into training and test set. We do not access the test data till the end. All data exploration and tuning is performed on the training set and by setting aside a small portion of training as a dev or validation set.

The following code snippet divides the data into training and test sets.

In [8]:
from sklearn.model_selection import train_test_split
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, random_state=42)

In [9]:
# let's examine the shapes of training and test sets
print ("# training samples:", train_features.shape[0])
print("# test samples:", test_features.shape[0])

# training samples: 15480
# test samples: 5160


It's time to perform another sanity check-here we check if the training feature matrix has the same number of rows as the training label vector. We perform the same check on test set too.

In [10]:
assert (train_features.shape[0] == train_labels.shape[0])
assert (test_features.shape[0] == test_labels.shape[0])

## 3.2 Pipeline: preprocessing + model

As a first step, build linear regression models with default parameter setting of `LinearRegression` APIs.
> We will make use of `Pipeline` API for combining data preprocessing and model building.

We will use `StandardScaler` feature scaling to bring all features on the same scale followed by a `LinearRegression` model.
The `pipeline` object has two components:
   1. `StandardScaler`  as step 1
   2. `LinearRegression` as step 2
   
After constructing the pipeline object, let's train it with training set.

In [11]:
# Set up the linear regression model.
lin_reg_pipeline = Pipeline([("feature scaling", StandardScaler()),
                             ("lin_reg", LinearRegression())])

# train linear regression model with normal equation.
lin_reg_pipeline.fit(train_features, train_labels)

Pipeline(steps=[('feature scaling', StandardScaler()),
                ('lin_reg', LinearRegression())])

Now that we have trained the model, let's check the learnt/estimated weight vectors (`intercept_` and `coef_`).

In [12]:
print("intercept (w_0):", lin_reg_pipeline[-1].intercept_)
print("weight vector (w_1, ... , w_m):", lin_reg_pipeline[-1].coef_)

intercept (w_0): 2.0703489205426377
weight vector (w_1, ... , w_m): [ 0.85210815  0.12065533 -0.30210555  0.34860575 -0.00164465 -0.04116356
 -0.89314697 -0.86784046]


A couple of things to notice:
* We accessed the LinearRegression object as `lin_reg_pipeline[-1]` which is the last step in the pipeline.
* The intercept can be obtained via `intercept_` member variable and 
* The weight vector corresponding to features via `coef_`

# STEP #4: Model Evaluation

## `Score`

Let's use `score` method to obtain train and test errors with twin objectives
> * Estimation of model performance as provided by test errror.
> * Comparison of errors for model diagnostic purpose(under/over/just right fit)

In [13]:
# evaluate model performance on the test set.
test_score = lin_reg_pipeline.score(test_features, test_labels)
print("Model performance on test set: ", test_score)

train_score = lin_reg_pipeline.score(train_features, 
                                    train_labels)
print("Model performance on train set: ", train_score)

Model performance on test set:  0.5910509795491351
Model performance on train set:  0.609873031052925


The `score` method returns `r2` score whose best value is 1. The `r2` scores on training and test are comparable but they are not that high. It points to underfitting issue in model training.